### Analysis V2
* AT&T and Verizon maps are done. 
* While being blocked with T-Mobile, I wanted to just check out how `utilities.load_unique_routes_df()` would work against the two finished maps. 
* Unfortunately I am blocked...Although testing  a few areas/routes with AT&T's map worked in the `multi_county_test.ipynb` file worked well, I ran into errors while working with certain routes
    * See section with HELP!

In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely.wkt
import utilities
from calitp import *
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Load in Provider Maps

In [2]:
verizon = utilities.load_verizon()

In [3]:
att = utilities.load_att()

In [4]:
tmobile = utilities.load_tmobile()

In [5]:
# Test simplify the geometry
# att2 = att.copy()
# att2["geometry"] = att2.geometry.simplify(tolerance=25)

In [6]:
# Geometries are true across each gdf
# verizon.geometry.is_valid.all(), att.geometry.is_valid.all(),tmobile.geometry.is_valid.all(), 

### Load in Routes 

In [7]:
unique_routes = utilities.load_unique_routes_df()

In [8]:
len(unique_routes)

2829

In [9]:
# Ensure CRS are the same
unique_routes.crs == att.crs == verizon.crs == tmobile.crs

True

#### Help! 
* Verizon doesn't work with any rouets: "# TypeError: `keep_geom_type` does not support GeometryCollection."
* Getting an error when using the `att2` gdf which simplifies geometry.
    * `TopologyException: Input geom 1 is invalid: Hole lies outside shell at -124.1036 40.437482000000003`
    *  Works fine with original att df.
* Certain routes like 'Sacramento 569  Amador Regional Transit System' and 'Upcountry 570  Amador Regional Transit System', cause errors 

In [10]:
# Routes from analysis v1 that didn't make sense
routes_wanted = [
    "Edmund D. Edelman Children’s Court Shuttle",
    "Oakland - San Francisco",
    "Foster City - Hillsdale Mall",
    "24 hour service daily",
    "Mammoth Lakes HWY 120E/395",
    "Avocado Heights/Bassett/West Valinda Shuttle",
    "Long Beach/San Pedro"
]

In [ ]:
# Doesn't work with geometry simplify?
# intersect_subset2 = gpd.overlay(unique_routes_subset, att2, how="intersection")

In [11]:
unique_routes_subset = unique_routes[unique_routes["route_name"].isin(routes_wanted)]

In [ ]:
# intersect_att = gpd.overlay(att, unique_routes_subset, how="intersection")

In [ ]:
# intersect_tmobile = gpd.overlay(tmobile, unique_routes_subset, how="intersection")

In [ ]:
# TypeError: `keep_geom_type` does not support GeometryCollection.
# intersect_verizon = gpd.overlay(verizon, unique_routes_subset, how="intersection")

In [20]:
problematic_route = unique_routes[unique_routes["long_route_name"] == 'Sacramento 569  Amador Regional Transit System']

In [26]:
test_att1 =  gpd.overlay(att, problematic_route, how="intersection", keep_geom_type = False)

In [30]:
problematic_route2 = unique_routes[unique_routes["long_route_name"] == 'Upcountry 570  Amador Regional Transit System']

In [31]:
test_att2 =  gpd.overlay(att, problematic_route2, how="intersection", keep_geom_type = False)

In [35]:
test_att2.assign(
        route_length=test_att2.geometry.to_crs(geography_utils.CA_StatePlane).length
    )

,itp_id,route_id,route_name,agency,original_route_length,long_route_name,geometry,route_length
0,11,570,Upcountry,Amador Regional Transit System,165618.801194,Upcountry 570 Amador Regional Transit System,"MULTILINESTRING ((-120.71725 38.36664, -120.71...",41914.614246
1,11,570,Upcountry,Amador Regional Transit System,165618.801194,Upcountry 570 Amador Regional Transit System,"MULTILINESTRING ((-120.51198 38.49178, -120.51...",139843.420436
2,11,570,Upcountry,Amador Regional Transit System,165618.801194,Upcountry 570 Amador Regional Transit System,"MULTILINESTRING ((-120.71725 38.36664, -120.71...",41914.614246
3,11,570,Upcountry,Amador Regional Transit System,165618.801194,Upcountry 570 Amador Regional Transit System,"MULTILINESTRING ((-120.51198 38.49178, -120.51...",97928.806190
4,11,570,Upcountry,Amador Regional Transit System,165618.801194,Upcountry 570 Amador Regional Transit System,"MULTILINESTRING ((-120.51198 38.49178, -120.51...",97928.806190


In [33]:
gpd.overlay(att, problematic_route2, how="intersection", keep_geom_type = True)

,itp_id,route_id,route_name,agency,original_route_length,long_route_name,geometry


In [ ]:
def split_routes():
    unique_routes = utilities.load_unique_routes_df()
    # Split unique_routes to 4 different ones
    # https://stackoverflow.com/questions/17315737/split-a-large-pandas-dataframe
    df_split = np.array_split(unique_routes, 4)
    
    df1 = df_split[0]
    df2 = df_split[1]
    df3 = df_split[2]
    df4 = df_split[3]
    
    return df1,df2,df3,df4


In [ ]:
routes1,routes2,routes3,routes4 = split_routes()

In [ ]:
def overlay_provider_unique_routes(provider: gpd.GeoDataFrame, 
                                  routes: gpd.GeoDataFrame,
                                  provider_name:str):
    
    # Empty dataframe to hold results
    all_intersected_routes = pd.DataFrame()
    
    # List of long route names
    unique_routes_list = routes.long_route_name.unique().tolist()
    
    # Intersect route by route, skipping those that don't work
    for i in unique_routes_list:
        filtered  = routes[routes.long_route_name == i].reset_index(drop=True)
        try: 
            intersected = gpd.overlay(provider, filtered, how="intersection", keep_geom_type = False)
            all_intersected_routes  = pd.concat([all_intersected_routes, intersected], axis=0)
        except: 
            pass 
            print(f"Didn't work: {i}")
    
    # utils.geoparquet_gcs_export(intersected_routes, utilities.GCS_FILE_PATH, f"{provider_name}_intersected_routes")
    
    return all_intersected_routes

In [ ]:
test_att = overlay_provider_unique_routes(att, routes1, 'att')